In [1]:
import os
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

In [2]:
import functools

import jax
import numpy as np
import scanpy as sc

from cfp.metrics import compute_mean_metrics, compute_metrics, compute_metrics_fast
import cfp.preprocessing as cfpp

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
split = 4

In [4]:
adata_preds = sc.read(f"/lustre/groups/ml01/workspace/ot_perturbation/models/chemcpa/combosciplex/adata_with_predictions_{split}.h5ad")

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
adata_train_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_{split}.h5ad"
adata_test_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_{split}.h5ad"
adata_ood_path = f"/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_{split}.h5ad"

In [6]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [7]:
adata_ref_test = adata_test[adata_test.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_test, n_comps=10)

adata_ref_ood = adata_ood[adata_ood.obs["condition"]!="control"].copy()
cfpp.centered_pca(adata_ref_ood, n_comps=10)

In [8]:
adata_pred_test = adata_preds[adata_preds.obs["split"]=="test"]
adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_test, ref_adata=adata_ref_test, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_test, ref_adata=adata_ref_test)
test_data_target_encoded = {}
test_data_target_decoded = {}
test_data_target_encoded_predicted = {}
test_data_target_decoded_predicted = {}
for cond in adata_test.obs["condition"].cat.categories:
    if cond == "control":
        continue
    test_data_target_encoded[cond] = adata_test[adata_test.obs["condition"] == cond].obsm["X_pca"]
    test_data_target_decoded[cond] = adata_test[adata_test.obs["condition"] == cond].X.toarray()
    test_data_target_decoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["CPA_pred"]
    test_data_target_encoded_predicted[cond] = adata_pred_test[adata_pred_test.obs["condition"] == cond].obsm["X_pca"]

adata_pred_ood = adata_preds[adata_preds.obs["split"]=="ood"]
adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]
cfpp.project_pca(query_adata=adata_pred_ood, ref_adata=adata_ref_ood, layer="CPA_PRED")
cfpp.project_pca(query_adata=adata_ood, ref_adata=adata_ref_ood)
ood_data_target_encoded = {}
ood_data_target_decoded = {}
ood_data_target_encoded_predicted = {}
ood_data_target_decoded_predicted = {}
for cond in adata_ood.obs["condition"].cat.categories:
    if cond == "control":
        continue
    ood_data_target_encoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].obsm["X_pca"]
    ood_data_target_decoded[cond] = adata_ood[adata_ood.obs["condition"] == cond].X.toarray()
    ood_data_target_decoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["CPA_pred"]
    ood_data_target_encoded_predicted[cond] = adata_pred_ood[adata_pred_ood.obs["condition"] == cond].obsm["X_pca"]

/tmp/ipykernel_1429101/2350350615.py:2: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_test.layers["CPA_PRED"] = adata_pred_test.obsm["CPA_pred"]
/tmp/ipykernel_1429101/2350350615.py:18: ImplicitModificationWarning: Setting element `.layers['CPA_PRED']` of view, initializing view as actual.
  adata_pred_ood.layers["CPA_PRED"] = adata_pred_ood.obsm["CPA_pred"]


In [9]:
test_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in test_data_target_decoded_predicted.keys()
}
ood_deg_dict = {
    k: v
    for k, v in adata_train.uns["rank_genes_groups_cov_all"].items()
    if k in ood_data_target_decoded_predicted.keys()
}

In [10]:
def get_mask(x, y):
    return x[:, [gene in y for gene in adata_train.var_names]]


ood_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, ood_data_target_decoded_predicted, ood_deg_dict)
ood_deg_target_decoded = jax.tree_util.tree_map(get_mask, ood_data_target_decoded, ood_deg_dict)

test_deg_target_decoded_predicted = jax.tree_util.tree_map(get_mask, test_data_target_decoded_predicted, test_deg_dict)
test_deg_target_decoded = jax.tree_util.tree_map(get_mask, test_data_target_decoded, test_deg_dict)


In [11]:
deg_ood_metrics = jax.tree_util.tree_map(compute_metrics, ood_deg_target_decoded, ood_deg_target_decoded_predicted)
deg_mean_ood_metrics = compute_mean_metrics(deg_ood_metrics, prefix="deg_ood_")

deg_test_metrics = jax.tree_util.tree_map(compute_metrics, test_deg_target_decoded, test_deg_target_decoded_predicted)
deg_mean_test_metrics = compute_mean_metrics(deg_test_metrics, prefix="deg_test_")


In [12]:
test_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, test_data_target_encoded, test_data_target_encoded_predicted
)
mean_test_metrics_encoded = compute_mean_metrics(test_metrics_encoded, prefix="encoded_test_")

test_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics_fast, test_data_target_decoded, test_data_target_decoded_predicted
)
mean_test_metrics_decoded = compute_mean_metrics(test_metrics_decoded, prefix="decoded_test_")

In [13]:
ood_metrics_encoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_encoded, ood_data_target_encoded_predicted
)
mean_ood_metrics_encoded = compute_mean_metrics(ood_metrics_encoded, prefix="encoded_ood_")

ood_metrics_decoded = jax.tree_util.tree_map(
    compute_metrics, ood_data_target_decoded, ood_data_target_decoded_predicted
)
mean_ood_metrics_decoded = compute_mean_metrics(ood_metrics_decoded, prefix="decoded_ood_")

In [14]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9566347102324168,
 'decoded_ood_sinkhorn_div_1': 81.87399800618489,
 'decoded_ood_sinkhorn_div_10': 48.286797841389976,
 'decoded_ood_sinkhorn_div_100': 4.176709493001302,
 'decoded_ood_e_distance': 7.687277621676658,
 'decoded_ood_mmd': 0.3303666164477666}

In [15]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7340767359733582,
 'decoded_test_e_distance': 52.207657303442566,
 'decoded_test_mmd_distance': 0.35693171977996824}

In [16]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9566347102324168,
 'decoded_ood_sinkhorn_div_1': 81.87399800618489,
 'decoded_ood_sinkhorn_div_10': 48.286797841389976,
 'decoded_ood_sinkhorn_div_100': 4.176709493001302,
 'decoded_ood_e_distance': 7.687277621676658,
 'decoded_ood_mmd': 0.3303666164477666}

In [17]:
mean_ood_metrics_encoded

{'encoded_ood_r_squared': -0.6940145591894785,
 'encoded_ood_sinkhorn_div_1': 8.70498243967692,
 'encoded_ood_sinkhorn_div_10': 4.159910996754964,
 'encoded_ood_sinkhorn_div_100': 3.0100749333699546,
 'encoded_ood_e_distance': 5.75464572793785,
 'encoded_ood_mmd': 0.23952009280522665}

In [18]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9361942807833353,
 'deg_ood_sinkhorn_div_1': 8.313067356745401,
 'deg_ood_sinkhorn_div_10': 2.3001553217569985,
 'deg_ood_sinkhorn_div_100': 1.9704030354817708,
 'deg_ood_e_distance': 3.871561367744386,
 'deg_ood_mmd': 0.33902732531229657}

In [19]:
deg_mean_test_metrics

{'deg_test_r_squared': 0.5326055335998535,
 'deg_test_sinkhorn_div_1': 25.30959072113037,
 'deg_test_sinkhorn_div_10': 16.308010864257813,
 'deg_test_sinkhorn_div_100': 15.088708381652832,
 'deg_test_e_distance': 30.00355805045886,
 'deg_test_mmd': 0.3935843336582184}

In [20]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7340767359733582,
 'decoded_test_e_distance': 52.207657303442566,
 'decoded_test_mmd_distance': 0.35693171977996824}

In [21]:
mean_ood_metrics_decoded

{'decoded_ood_r_squared': 0.9566347102324168,
 'decoded_ood_sinkhorn_div_1': 81.87399800618489,
 'decoded_ood_sinkhorn_div_10': 48.286797841389976,
 'decoded_ood_sinkhorn_div_100': 4.176709493001302,
 'decoded_ood_e_distance': 7.687277621676658,
 'decoded_ood_mmd': 0.3303666164477666}

In [22]:
deg_mean_ood_metrics

{'deg_ood_r_squared': 0.9361942807833353,
 'deg_ood_sinkhorn_div_1': 8.313067356745401,
 'deg_ood_sinkhorn_div_10': 2.3001553217569985,
 'deg_ood_sinkhorn_div_100': 1.9704030354817708,
 'deg_ood_e_distance': 3.871561367744386,
 'deg_ood_mmd': 0.33902732531229657}

In [23]:
mean_test_metrics_decoded

{'decoded_test_r_squared': 0.7340767359733582,
 'decoded_test_e_distance': 52.207657303442566,
 'decoded_test_mmd_distance': 0.35693171977996824}

In [24]:
mean_test_metrics_encoded

{'encoded_test_r_squared': -1.0442973344820967,
 'encoded_test_sinkhorn_div_1': 29.692364940643312,
 'encoded_test_sinkhorn_div_10': 25.478276634216307,
 'encoded_test_sinkhorn_div_100': 24.132667961120607,
 'encoded_test_e_distance': 47.95706470009132,
 'encoded_test_mmd': 0.3508694690465927}

In [25]:
ood_metrics_encoded

{'Cediranib+PCI-34051': {'r_squared': 0.2974282503128052,
  'sinkhorn_div_1': 7.901453495025635,
  'sinkhorn_div_10': 3.373772621154785,
  'sinkhorn_div_100': 2.2574081420898438,
  'e_distance': 4.2643744085450805,
  'mmd': 0.23257583},
 'Givinostat+Crizotinib': {'r_squared': -1.573023796081543,
  'sinkhorn_div_1': 7.771107196807861,
  'sinkhorn_div_10': 3.1570863723754883,
  'sinkhorn_div_100': 1.9116382598876953,
  'e_distance': 3.527157821346189,
  'mmd': 0.22273003},
 'Givinostat+SRT2104': {'r_squared': -0.8988072872161865,
  'sinkhorn_div_1': 7.2082109451293945,
  'sinkhorn_div_10': 2.5161962509155273,
  'sinkhorn_div_100': 1.2903375625610352,
  'e_distance': 2.289647721892873,
  'mmd': 0.22127233},
 'Givinostat+Sorafenib': {'r_squared': -0.9011318683624268,
  'sinkhorn_div_1': 14.161542892456055,
  'sinkhorn_div_10': 10.284482955932617,
  'sinkhorn_div_100': 9.452860832214355,
  'e_distance': 18.726136956131363,
  'mmd': 0.30729485},
 'control+Givinostat': {'r_squared': -1.340876

In [26]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/results/chemcpa"

In [27]:
import os
import pandas as pd

pd.DataFrame.from_dict(ood_metrics_encoded).to_csv(os.path.join(output_dir, f"ood_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(ood_metrics_decoded).to_csv(os.path.join(output_dir, f"ood_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_encoded).to_csv(os.path.join(output_dir, f"test_metrics_encoded_{split}.csv"))
pd.DataFrame.from_dict(test_metrics_decoded).to_csv(os.path.join(output_dir, f"test_metrics_decoded_{split}.csv"))
pd.DataFrame.from_dict(deg_test_metrics).to_csv(os.path.join(output_dir, f"test_metrics_deg_{split}.csv"))
pd.DataFrame.from_dict(deg_ood_metrics).to_csv(os.path.join(output_dir, f"ood_metrics_ood_{split}.csv"))
